In [1]:
import gsn_db
import gsn_datacollector
import bid_operator

import datetime
import pandas as pd
import numpy as np
import json
import math

In [2]:
DATABASE = "dev_gsn"
BIDDING_INDEX = {
    'cpc': 'cpc_bid',
    'Target CPA': 'cpa_bid',
}
DESTINATION_INDEX = {
    'cpc': 'clicks',
    'cpa': 'conversions',
    'LINK_CLICKS': 'clicks',
    'CONVERSIONS':'conversions',
}

In [3]:
class Field:
    ai_start_date = 'ai_start_date'
    ai_stop_date = 'ai_stop_date'
    ad_group_id = 'adgroup_id'
    campaign_id = 'campaign_id'
    keyword_id = 'keyword_id'
    target = 'target'
    bid_amount = 'bid_amount'
    target_left = 'target_left'
    init_bid = 'init_bid'
    last_bid = 'last_bid'
    keyword_progress = 'keyword_progress'
    campaign_progress = 'campaign_progress'
    period = 'period'

In [4]:
class CampaignAdapter(object):
    def __init__(self, campaign_id):
        self.mydb = gsn_db.connectDB( DATABASE )
        self.hour_per_day = 24
        self.campaign_id = campaign_id
        self.request_time = datetime.datetime.now()
        self.time_progress = ( self.request_time.hour + 1 ) / self.hour_per_day
        self.init_bid_dict = dict()
        self.last_bid_dict = dict()
        
    def _get_df(self):
        campaign_sql = "SELECT * FROM campaign_target WHERE campaign_id={}".format( self.campaign_id )
        keyword_sql = "select * from keywords_insights WHERE campaign_id = {} AND DATE(request_time) = '{}'".format( self.campaign_id, self.request_time.date() )
        self.df_camp = pd.read_sql( campaign_sql, con=self.mydb )
        self.df_keyword = pd.read_sql( keyword_sql, con=self.mydb )
        return
    
    def get_keyword_id_list(self):
        keyword_id_list_sql = "select DISTINCT keyword_id from (select * from keywords_insights WHERE campaign_id = {} and status='enabled' order by request_time) as a group by keyword_id".format( self.campaign_id )
        self.mycursor = self.mydb.cursor()
        self.mycursor.execute( keyword_id_list_sql )
        default = self.mycursor.fetchall()
        self.keyword_id_list = [ i[0] for i in default ]
        return self.keyword_id_list
    
    def _get_bid(self):
        df_init_bid = pd.read_sql( "SELECT * FROM adgroup_initial_bid WHERE campaign_id={} ;".format( self.campaign_id ), con=self.mydb )
        self.get_keyword_id_list()
        bid_amount_type = BIDDING_INDEX[ self.df_keyword['bidding_type'].iloc[0] ]
        for keyword_id in self.keyword_id_list:
            if len(self.df_keyword[self.df_keyword.keyword_id==keyword_id]) != 0:
                init_bid = df_init_bid[Field.bid_amount][df_init_bid.keyword_id==keyword_id].head(1).iloc[0].astype(dtype=object)
                last_bid = self.df_keyword[ bid_amount_type ][self.df_keyword.keyword_id==keyword_id].tail(1).iloc[0].astype(dtype=object)
                self.init_bid_dict.update({ keyword_id: init_bid })
                self.last_bid_dict.update({ keyword_id: last_bid })
        return
        
    def get_periods_left(self):
        self.periods_left = ( self.df_camp[ Field.ai_stop_date ].iloc[0] - self.request_time.date() ).days + 1
        return self.periods_left
    
    def get_periods(self):
        self.periods = ( self.df_camp[ Field.ai_stop_date ].iloc[0] - self.df_camp[ Field.ai_start_date ].iloc[0] ).days
        return self.periods
    
    def get_campaign_performance(self):
        self.campaign_performance = self.df_camp[ Field.target ].div(self.df_camp[ Field.period ] ).sum()
        return self.campaign_performance
    
    def get_campaign_target(self):
        self.campaign_target = self.df_camp[ Field.target_left ].iloc[0].astype(dtype=object)
        return self.campaign_target
    
    def get_campaign_day_target(self):
        self.campaign_day_target = self.campaign_target / self.periods_left
        return self.campaign_day_target

    def get_campaign_progress(self):
        self.campaign_progress = self.campaign_performance / self.campaign_day_target
        self.campaign_progress = 1 if self.campaign_day_target <= 0 else self.campaign_progress
        return self.campaign_progress
        
    def retrieve_campaign_attribute(self):
        self._get_df()
        self.get_keyword_id_list()
        self._get_bid()
        self.get_periods_left()
        self.get_periods()
        self.get_campaign_performance()
        self.get_campaign_target()
        self.get_campaign_day_target()
        self.get_campaign_progress()
        self.mydb.close()
        return

In [5]:
class KeywordGroupAdapter(object):
    def __init__(self, keyword_id, camp):
        self.mydb = gsn_db.connectDB( DATABASE )
        self.keyword_id = keyword_id
        self.camp = camp
    
    def get_campaign_id(self):
        self.campaign_id = self.camp.df_keyword[ Field.campaign_id ].iloc[0].astype(dtype=object)
        return self.campaign_id
    
    def get_keyword_day_target(self):
        keyword_num = len( self.camp.keyword_id_list )
        self.keyword_day_target = self.camp.campaign_day_target / keyword_num
        return self.keyword_day_target
    
    def get_keyword_performance(self):
        try:
            target_performance_index = DESTINATION_INDEX[self.camp.df_camp['destination_type'].iloc[0]]
            self.keyword_performance = self.camp.df_keyword[self.camp.df_keyword.keyword_id==self.keyword_id][[ target_performance_index ]].tail(1).iloc[0,0]
        except Exception as e:
            print('[gsn_adapter.KeywordGroupAdapter.get_keyword_performance()]', e)
            self.keyword_performance = 0
        if math.isnan(self.keyword_performance):
            self.keyword_performance = 0
        return self.keyword_performance
    
    def get_bid(self):
        self.init_bid = self.camp.init_bid_dict[self.keyword_id]
        self.last_bid = self.camp.last_bid_dict[self.keyword_id]
        return
    
    def get_keyword_time_target(self):
        self.keyword_time_target = self.keyword_day_target * self.camp.time_progress
        return self.keyword_time_target
    
    def get_keyword_progress(self):
#         print(self.keyword_performance, self.keyword_time_target)
        self.keyword_progress = self.keyword_performance / self.keyword_time_target
        self.keyword_progress = 1 if self.keyword_time_target <= 0 else self.keyword_progress
        return self.keyword_progress
    
    def retrieve_keyword_attribute(self):
        self.get_campaign_id()
        self.get_keyword_day_target()
        self.get_keyword_performance()
        self.get_bid()
        self.get_keyword_time_target()
        self.get_keyword_progress()
        self.mydb.close()
        return {
            Field.keyword_id:self.keyword_id,
            Field.init_bid:self.init_bid,
            Field.last_bid:self.last_bid,
            Field.keyword_progress:self.keyword_progress,
            Field.campaign_progress:self.camp.campaign_progress
        }

In [6]:

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)
        
def main():
    start_time = datetime.datetime.now()
    campaign_id_list = gsn_db.get_campaign()['campaign_id'].tolist()
    print('[on-going campaign_id_list]: ', campaign_id_list)
    for campaign_id in campaign_id_list:
        print('[campaign_id]: ', campaign_id)
        result={ 'media': 'GSN', 'campaign_id': campaign_id, 'contents':[] }
        release_version_result = {  }
#         try:
        camp = CampaignAdapter( campaign_id )
        camp.retrieve_campaign_attribute()
        destination_type = camp.df_camp['destination_type'].iloc[0]
        account_id = camp.df_camp['customer_id'].iloc[0]
        for keyword_id in camp.keyword_id_list:
            print('[keyword_id]: ', keyword_id)
            try:
                keyword_group = KeywordGroupAdapter( keyword_id, camp )
                status_dict = keyword_group.retrieve_keyword_attribute()
                media = result['media']
                bid_dict = bid_operator.adjust(media, **status_dict)
                datacollect_keyword_group = gsn_datacollector.KeywordGroup(
                    customer_id=account_id, campaign_id=campaign_id, keyword_id=keyword_id)
                datacollect_keyword_group.get_keyword_criterion()
                datacollect_keyword_group.update_bid(bid_micro_amount=bid_dict['bid'])
                result['contents'].append(bid_dict)
                del keyword_group
            except Exception as e:
                print('[gsn_adapter.KeywordGroupAdapter] update unavailable: ', e)
        
        mydict_json = json.dumps(result, cls=MyEncoder)
        release_json = json.dumps(release_version_result)
        gsn_db.insert_result( campaign_id, mydict_json )
        del camp
#         except:
#             print('pass')
#             pass
        
#     campaign_id = 1747836664
#     result={ 'media': 'GSN', 'campaign_id': campaign_id, 'contents':[] }
#     camp = CampaignAdapter( campaign_id )
#     camp.retrieve_campaign_attribute()
#     keyword_list = camp.get_keyword_list()
#     for keyword in keyword_list:
#         s = AdGroupAdapter( keyword, camp )
#         status = s.retrieve_keyword_attribute()
#         media = result['media']
#         bid = bid_operator.adjust(media, **status)
#         result['contents'].append(bid)
#         print(result)
#         del s
#     del camp
    
    print(datetime.datetime.now()-start_time)
    return

In [7]:
if __name__=='__main__':
    main()

[on-going campaign_id_list]:  [2030591888, 2031012661, 2031455940]
[campaign_id]:  2030591888


/home/tim_su/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: RuntimeWarning: divide by zero encountered in double_scalars
/home/tim_su/.local/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


[keyword_id]:  10113686
{'keyword_id': 10113686, 'init_bid': 8, 'last_bid': 11.2, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 10113686, 'bid': 8.0}
[keyword_id]:  295108130630
{'keyword_id': 295108130630, 'init_bid': 8, 'last_bid': 11.2, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 295108130630, 'bid': 8.0}
[keyword_id]:  299877466065
{'keyword_id': 299877466065, 'init_bid': 7, 'last_bid': 7.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 299877466065, 'bid': 7.0}


/home/tim_su/.local/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in true_divide


[keyword_id]:  306595805486
{'keyword_id': 306595805486, 'init_bid': 8, 'last_bid': 8.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 306595805486, 'bid': 8.0}
[keyword_id]:  306595805686
{'keyword_id': 306595805686, 'init_bid': 8, 'last_bid': 8.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 306595805686, 'bid': 8.0}
[keyword_id]:  311037542867
{'keyword_id': 311037542867, 'init_bid': 8, 'last_bid': 11.2, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 311037542867, 'bid': 8.0}
[keyword_id]:  312586315670
{'keyword_id': 312586315670, 'init_bid': 8, 'last_bid': 11.2, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 312586315670, 'bid': 8.0}
[keyword_id]:  312586316830
{'keyword_id': 312586316830, 'init_bid': 8, 'last_bid': 11.2, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 312586316830, 'bid': 8.0}
[keyword_id]:  312586316870
{'keyword_id': 312586316870, 'init_bid': 8, 'last_bid': 11.2, 'keyword_progress': 1,

[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  19429611
{'keyword_id': 19429611, 'init_bid': 8, 'last_bid': 10.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 19429611, 'bid': 8.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  19505073
{'keyword_id': 19505073, 'init_bid': 7, 'last_bid': 8.55, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 19505073, 'bid': 7.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  19943741
{'keyword_id': 19943741, 'init_bid': 8, 'last_bid': 10.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 19943741, 'bid': 8.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  20029626
{'keyword_id': 20029626, 'init_bid': 7, 'last_bid': 10.0, 'keyword_progress': 1, 'campaign_progress': 

[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  111292198877
{'keyword_id': 111292198877, 'init_bid': 7, 'last_bid': 8.55, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 111292198877, 'bid': 7.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  296549800051
{'keyword_id': 296549800051, 'init_bid': 8, 'last_bid': 8.55, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 296549800051, 'bid': 8.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  296627029673
{'keyword_id': 296627029673, 'init_bid': 8, 'last_bid': 8.55, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 296627029673, 'bid': 8.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  296771498339
{'keyword_id': 296771498339, 'init_bid': 7, 'last_bid': 10.0, 

[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  474164644523
{'keyword_id': 474164644523, 'init_bid': 7, 'last_bid': 8.55, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 474164644523, 'bid': 7.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  489343770232
{'keyword_id': 489343770232, 'init_bid': 7, 'last_bid': 8.55, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 489343770232, 'bid': 7.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  567395418205
{'keyword_id': 567395418205, 'init_bid': 8, 'last_bid': 10.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 567395418205, 'bid': 8.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  633400308162
{'keyword_id': 633400308162, 'init_bid': 8, 'last_bid': 10.0, 

[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  295053599496
{'keyword_id': 295053599496, 'init_bid': 8, 'last_bid': 4.96, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 295053599496, 'bid': 8.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  295439763096
{'keyword_id': 295439763096, 'init_bid': 8, 'last_bid': 10.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 295439763096, 'bid': 8.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  296481161000
{'keyword_id': 296481161000, 'init_bid': 7, 'last_bid': 10.0, 'keyword_progress': 1, 'campaign_progress': 1}
{'keyword_id': 296481161000, 'bid': 7.0}
[gsn_adapter.KeywordGroupAdapter] update unavailable:  'NoneType' object is not subscriptable
[keyword_id]:  296694167480
{'keyword_id': 296694167480, 'init_bid': 7, 'last_bid': 9.3, '

In [8]:
#!jupyter nbconvert --to script gsn_adapter.ipynb

[NbConvertApp] Converting notebook gsn_adapter.ipynb to script
[NbConvertApp] Writing 9798 bytes to gsn_adapter.py
